In [9]:
from pathlib import Path

import numpy as np
from PIL import Image
from tqdm import tqdm

In [10]:
npz_root = Path("./pred_npz_with_image")

In [12]:
def normalize(img, wl, ww):
    img = (img - wl + 0.5 * ww) / ww
    return img


npz_file_list = list(npz_root.glob("*.npz"))
npz_file_list = sorted(npz_file_list, key=lambda x: int(Path(x).stem))

save_png_root = Path("png")
save_png_root.mkdir(exist_ok=True)
for npz_file in tqdm(npz_file_list):
    save_png_path = save_png_root / npz_file.with_suffix(".png").name
    if save_png_path.exists():
        continue
    _data = np.load(npz_file)
    pred = _data["pred"]
    gt = _data["gt"]
    slice_num = _data["start_z"]
    axis_msk = _data["axis_msk"]
    pid = _data["Patient ID"]
    ww = _data["WW"]  # dicomのWindowWidthを使用
    wl = _data["WL"]  # dicomのWindowWidthを使用
    img = _data["img"]

    img = normalize(img, wl, ww)
    np.clip(img, 0, 1, out=img)
    img = (img * 255).astype(np.uint8)
    img = np.tile(img[..., None], [1, 1, 3])
    blend = 0.5

    img_all = img.copy()

    img_gt = img.copy()
    overlap = gt[slice_num] == 1
    color = np.array((255, 0, 0))
    img_gt[overlap] = blend * img[overlap] + (1 - blend) * color
    img_all[overlap] = blend * img[overlap] + (1 - blend) * color

    img_pred = img.copy()
    overlap = pred[slice_num] == 1
    color = np.array((0, 255, 0))
    img_pred[overlap] = blend * img[overlap] + (1 - blend) * color
    img_all[overlap] = blend * img[overlap] + (1 - blend) * color

    overlap = axis_msk[slice_num] == 1
    color = np.array((0, 0, 255))
    img[overlap] = blend * img[overlap] + (1 - blend) * color
    img_all[overlap] = blend * img[overlap] + (1 - blend) * color

    margin = 0
    out = np.zeros((img.shape[0] * 2 + margin, img.shape[1] * 2 + margin, 3), np.uint8)
    out[: img.shape[0], : img.shape[1]] = img
    out[: img.shape[0], img.shape[1] + margin : img.shape[1] * 2 + margin] = img_gt
    out[
        img.shape[0] + margin : img.shape[0] * 2 + margin,
        : img.shape[1],
    ] = img_pred
    out[
        img.shape[0] + margin : img.shape[0] * 2 + margin,
        img.shape[1] + margin : img.shape[1] * 2 + margin,
    ] = img_all

    out = Image.fromarray(out)
    out.save(save_png_path)
    
    
    

100%|██████████| 12/12 [00:02<00:00,  5.17it/s]


In [25]:
# --- 設定: 色をここで変更可能 ---
COLOR_GT   = np.array((238, 205, 28))  # GT → 赤
COLOR_PRED = np.array((33, 133, 207))  # 予測 → 緑
COLOR_AXIS = np.array((236, 115, 141))  # 直径(axis_msk) → 青
AXIS_THICKNESS = 3
BLEND = 0.1  # オーバーレイ時の透過率 (0=全部色, 1=全部画像)

# --- 正規化関数 ---
def normalize(img, wl, ww):
    img = (img - wl + 0.5 * ww) / ww
    return img

# --- 入出力フォルダ ---
npz_root = Path("./pred_npz_with_image")
npz_file_list = sorted(npz_root.glob("*.npz"), key=lambda x: int(Path(x).stem))

save_png_root = Path("png_next")
save_png_root.mkdir(exist_ok=True)

# --- メインループ ---
for npz_file in tqdm(npz_file_list, desc="Making PNGs"):
    save_png_path = save_png_root / npz_file.with_suffix(".png").name
    if save_png_path.exists():
        continue

    # .npz 読み込み
    _data = np.load(npz_file)
    pred = _data["pred"]
    gt = _data["gt"]
    slice_num = _data["start_z"]
    axis_msk = _data["axis_msk"]
    ww = _data["WW"]
    wl = _data["WL"]
    img = _data["img"]

    # --- 原画像正規化 ---
    img = normalize(img, wl, ww)
    np.clip(img, 0, 1, out=img)
    img = (img * 255).astype(np.uint8)
    img = np.tile(img[..., None], [1, 1, 3])  # Grayscale→RGB

    # --- 合成用コピー ---
    img_all = img.copy()

    # --- GTオーバーレイ ---
    overlap = gt[slice_num] == 1
    img_all[overlap] = BLEND * img[overlap] + (1 - BLEND) * COLOR_GT

    # --- Predオーバーレイ ---
    overlap = pred[slice_num] == 1
    img_all[overlap] = BLEND * img[overlap] + (1 - BLEND) * COLOR_PRED

    # --- Axisオーバーレイ ---
    overlap = axis_msk[slice_num] == 1
    #overlap = thicken_mask(overlap, thickness=3)  # ← 3ピクセル分太くする
    img_all[overlap] = BLEND * img[overlap] + (1 - BLEND) * COLOR_AXIS

    # --- 保存 (右下だけにしたので img_all 単独) ---
    out = Image.fromarray(img_all.astype(np.uint8))
    out.save(save_png_path)


Making PNGs: 100%|██████████| 12/12 [00:02<00:00,  5.43it/s]
